In [3]:
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('train_fill_na.csv')
test = pd.read_csv('test_fill_na.csv')
labels = pd.read_csv('labels.csv')

In [18]:
# train.shape, test.shape, labels.shape

# labels

# train

# train.isna().sum()

# test.isna().sum()

In [4]:
df = pd.concat([train, test], axis=0)

In [20]:
# df.isna().sum()

# train.shape[0]+test.shape[0]==df.shape[0]

# df

# sns.heatmap(pd.isnull(df))

# ### quick check of labels

# temp = pd.read_csv('train_s3TEQDk.csv')

# for i in range(temp.shape[0]):
#     if temp.iloc[i, -1] != labels.iloc[i,0]:
#         print(i)

# ### check passed ∞

# df

# df['Age'].skew()

# df['Vintage'].skew()

# df['Avg_Account_Balance'].skew()

# np.log(df['Avg_Account_Balance']).skew()

# np.log(df['Vintage']).skew()

# np.log(df['Age']).skew()

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

sc.fit_transform(df[['Avg_Account_Balance']]).reshape(1,-1)[0]

df['Avg_Account_Balance_Unskewed'] = np.log1p(df['Avg_Account_Balance'])

df['Age_scaled'] = sc.fit_transform(df[['Age']]).reshape(1,-1)[0]

df['Vintage_scaled'] = sc.fit_transform(df[['Vintage']]).reshape(1,-1)[0]

df_dum = pd.concat([df, pd.get_dummies(df[['Gender','Region_Code','Occupation','Channel_Code','Is_Active','Credit_Product']], drop_first=True)], axis=1)


df_dum.drop(['Gender','Region_Code','Occupation','Channel_Code','Is_Active','Credit_Product'], axis=1, inplace=True)

original = ['Age','Vintage','Avg_Account_Balance']
fe = ['Age_scaled', 'Vintage_scaled', 'Avg_Account_Balance_Unskewed']

In [6]:
df.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Avg_Account_Balance,Is_Active,Credit_Product,Avg_Account_Balance_Unskewed,Age_scaled,Vintage_scaled
0,Female,73,RG268,Other,X3,43,1045696,No,No,13.860194,1.963311,-0.121384
1,Female,30,RG277,Salaried,X1,32,581988,No,No,13.274207,-0.933890,-0.461633
2,Female,56,RG268,Self_Employed,X3,26,1484315,Yes,No,14.210465,0.817906,-0.647223
3,Male,34,RG270,Salaried,X1,19,470454,No,No,13.061456,-0.664383,-0.863745
4,Female,30,RG282,Salaried,X1,33,886787,No,No,13.695361,-0.933890,-0.430701


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
#le.fit_transform(df[['Gender','Region_Code','Occupation','Channel_Code','Is_Active','Credit_Product']]).reshape(1,-1)

In [10]:
df['Gender_encoded'] = le.fit_transform(df['Gender'])

In [11]:
df['Region_Code_encoded'] = le.fit_transform(df['Region_Code'])

In [12]:
df['Occupation_encoded'] = le.fit_transform(df['Occupation'])

In [13]:
df['Channel_Code_encoded'] = le.fit_transform(df['Channel_Code'])

In [14]:
df['Is_Active_encoded'] = le.fit_transform(df['Is_Active'])

In [15]:
df['Credit_Product_encoded'] = le.fit_transform(df['Credit_Product'])

In [16]:
df_code = df.drop(['Gender','Region_Code','Occupation','Channel_Code','Is_Active','Credit_Product'], axis=1)

In [17]:
df_code

,Age,Vintage,Avg_Account_Balance,Avg_Account_Balance_Unskewed,Age_scaled,Vintage_scaled,Gender_encoded,Region_Code_encoded,Occupation_encoded,Channel_Code_encoded,Is_Active_encoded,Credit_Product_encoded
0,73,43,1045696,13.860194,1.963311,-0.121384,0,18,1,2,0,0
1,30,32,581988,13.274207,-0.933890,-0.461633,0,27,2,0,0,0
2,56,26,1484315,14.210465,0.817906,-0.647223,0,18,3,2,1,0
3,34,19,470454,13.061456,-0.664383,-0.863745,1,20,2,0,0,0
4,30,33,886787,13.695361,-0.933890,-0.430701,0,32,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
105307,52,86,4242558,15.260677,0.548399,1.208681,1,18,2,1,1,1
105308,55,86,1159153,13.963201,0.750529,1.208681,1,27,1,1,0,1
105309,35,15,1703727,14.348329,-0.597006,-0.987472,1,4,2,3,0,0
105310,53,93,737178,13.510586,0.615775,1.425203,1,4,1,2,1,0


# MACHINE LEARNING

In [18]:
from sklearn.linear_model import LogisticRegression, LassoCV, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.decomposition import PCA
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier



In [22]:
from mlxtend.classifier import StackingCVClassifier

In [21]:
# import sys
# !{sys.executable} -m pip install mlxtend

In [23]:
lr = LogisticRegression()
sgdc = SGDClassifier()
ada = AdaBoostClassifier()
bag = BaggingClassifier()
etc = ExtraTreeClassifier()
gbc = GradientBoostingClassifier()
rfc = RandomForestClassifier()
gnb = GaussianNB()
bnb = BernoulliNB()
knn = KNeighborsClassifier()
mlpc = MLPClassifier()
xgb = XGBClassifier()
xgbr = XGBRFClassifier()
lgb = LGBMClassifier()
cat = CatBoostClassifier()

models = [lr, ada, bag, etc, gbc, rfc, 
           gnb, bnb, knn, xgb, xgbr, lgb, cat]


In [24]:
x_dum = df_dum.iloc[:len(train),:]
X_dum = df_dum.iloc[len(train):,:]

x = df_code.iloc[:len(train):,:]
X = df_code.iloc[len(train):,:]
y = labels

def run_model(model, data):
    model.fit(data[0], data[1])
    #pred = model.decision_function(data[2])
    pred_prob = model.predict_proba(data[2])
    #score = roc_auc_score(data[3], pred, )
    acc = roc_auc_score(data[3], pred_prob[:,1])
    return acc

## APPROACH 1
#### ORG DUM DATA

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_dum.drop(fe, axis=1), y, test_size=0.13, shuffle=True)

data = [x_train, y_train, x_test, y_test]


In [26]:
model_names = []
model_performances = []

for model in models:
    perf = run_model(model, data)
    model_performances.append(perf)
    model_names.append(str(model.get_params).split('(')[0].split()[-1])
    print(model_names[-1], model_performances[-1],
          '\n\n------------------------------------------------------------------\n\n')
    

LogisticRegression 0.44908058014494223 

------------------------------------------------------------------




KeyboardInterrupt: 

### Submission

In [66]:
x_train, x_test, y_train, y_test = train_test_split(x_dum.drop(['Age_scaled','Vintage_scaled','Avg_Account_Balance'], axis=1), y, test_size=0.13, shuffle=True)

data = [x_train, y_train, x_test, y_test]


In [67]:
model_names = []
model_performances = []

for model in models:
    perf = run_model(model, data)
    model_performances.append(perf)
    model_names.append(str(model.get_params).split('(')[0].split()[-1])
    print(model_names[-1], model_performances[-1],
          '\n\n------------------------------------------------------------------\n\n')
    

LogisticRegression 0.766966712260848 

------------------------------------------------------------------


AdaBoostClassifier 0.7873042072635354 

------------------------------------------------------------------


BaggingClassifier 0.7379622851755723 

------------------------------------------------------------------


ExtraTreeClassifier 0.6185897328727962 

------------------------------------------------------------------


GradientBoostingClassifier 0.797635296425195 

------------------------------------------------------------------


RandomForestClassifier 0.7372073324883235 

------------------------------------------------------------------


GaussianNB 0.6644715303861042 

------------------------------------------------------------------


BernoulliNB 0.7315316317349906 

------------------------------------------------------------------


KNeighborsClassifier 0.7178100326103852 

------------------------------------------------------------------


[18:56:43] WARNING: C:

126:	learn: 0.4283814	total: 5.58s	remaining: 38.3s
127:	learn: 0.4283495	total: 5.62s	remaining: 38.3s
128:	learn: 0.4283229	total: 5.68s	remaining: 38.3s
129:	learn: 0.4282966	total: 5.75s	remaining: 38.5s
130:	learn: 0.4282605	total: 5.79s	remaining: 38.4s
131:	learn: 0.4281953	total: 5.85s	remaining: 38.5s
132:	learn: 0.4281580	total: 5.92s	remaining: 38.6s
133:	learn: 0.4281193	total: 5.95s	remaining: 38.5s
134:	learn: 0.4280357	total: 5.99s	remaining: 38.4s
135:	learn: 0.4279980	total: 6.05s	remaining: 38.4s
136:	learn: 0.4279617	total: 6.08s	remaining: 38.3s
137:	learn: 0.4279322	total: 6.12s	remaining: 38.2s
138:	learn: 0.4279012	total: 6.15s	remaining: 38.1s
139:	learn: 0.4278640	total: 6.19s	remaining: 38s
140:	learn: 0.4278459	total: 6.23s	remaining: 38s
141:	learn: 0.4278112	total: 6.29s	remaining: 38s
142:	learn: 0.4277851	total: 6.33s	remaining: 37.9s
143:	learn: 0.4277379	total: 6.37s	remaining: 37.9s
144:	learn: 0.4277120	total: 6.43s	remaining: 37.9s
145:	learn: 0.4276

285:	learn: 0.4232557	total: 12.7s	remaining: 31.7s
286:	learn: 0.4232425	total: 12.7s	remaining: 31.6s
287:	learn: 0.4232064	total: 12.8s	remaining: 31.6s
288:	learn: 0.4231862	total: 12.8s	remaining: 31.5s
289:	learn: 0.4231577	total: 12.9s	remaining: 31.5s
290:	learn: 0.4231266	total: 12.9s	remaining: 31.5s
291:	learn: 0.4231043	total: 13s	remaining: 31.4s
292:	learn: 0.4230780	total: 13s	remaining: 31.3s
293:	learn: 0.4230523	total: 13s	remaining: 31.3s
294:	learn: 0.4230260	total: 13.1s	remaining: 31.3s
295:	learn: 0.4230087	total: 13.2s	remaining: 31.3s
296:	learn: 0.4229838	total: 13.2s	remaining: 31.2s
297:	learn: 0.4229683	total: 13.2s	remaining: 31.2s
298:	learn: 0.4229485	total: 13.3s	remaining: 31.2s
299:	learn: 0.4229295	total: 13.3s	remaining: 31.1s
300:	learn: 0.4228960	total: 13.4s	remaining: 31.1s
301:	learn: 0.4228797	total: 13.4s	remaining: 31s
302:	learn: 0.4228597	total: 13.5s	remaining: 31s
303:	learn: 0.4228370	total: 13.5s	remaining: 30.9s
304:	learn: 0.4228150	

446:	learn: 0.4194136	total: 19.9s	remaining: 24.7s
447:	learn: 0.4193975	total: 20s	remaining: 24.6s
448:	learn: 0.4193758	total: 20s	remaining: 24.6s
449:	learn: 0.4193587	total: 20.1s	remaining: 24.5s
450:	learn: 0.4193399	total: 20.1s	remaining: 24.5s
451:	learn: 0.4193056	total: 20.2s	remaining: 24.4s
452:	learn: 0.4192824	total: 20.2s	remaining: 24.4s
453:	learn: 0.4192675	total: 20.2s	remaining: 24.3s
454:	learn: 0.4192499	total: 20.3s	remaining: 24.3s
455:	learn: 0.4192220	total: 20.4s	remaining: 24.3s
456:	learn: 0.4191982	total: 20.4s	remaining: 24.2s
457:	learn: 0.4191767	total: 20.5s	remaining: 24.2s
458:	learn: 0.4191569	total: 20.5s	remaining: 24.2s
459:	learn: 0.4191408	total: 20.6s	remaining: 24.1s
460:	learn: 0.4191324	total: 20.6s	remaining: 24.1s
461:	learn: 0.4191147	total: 20.6s	remaining: 24s
462:	learn: 0.4190821	total: 20.7s	remaining: 24s
463:	learn: 0.4190671	total: 20.7s	remaining: 23.9s
464:	learn: 0.4190365	total: 20.8s	remaining: 23.9s
465:	learn: 0.419020

609:	learn: 0.4161075	total: 27.4s	remaining: 17.5s
610:	learn: 0.4160833	total: 27.4s	remaining: 17.5s
611:	learn: 0.4160653	total: 27.5s	remaining: 17.4s
612:	learn: 0.4160474	total: 27.5s	remaining: 17.4s
613:	learn: 0.4160314	total: 27.5s	remaining: 17.3s
614:	learn: 0.4160148	total: 27.6s	remaining: 17.3s
615:	learn: 0.4159974	total: 27.6s	remaining: 17.2s
616:	learn: 0.4159815	total: 27.7s	remaining: 17.2s
617:	learn: 0.4159632	total: 27.7s	remaining: 17.1s
618:	learn: 0.4159485	total: 27.8s	remaining: 17.1s
619:	learn: 0.4159363	total: 27.8s	remaining: 17s
620:	learn: 0.4159192	total: 27.8s	remaining: 17s
621:	learn: 0.4158975	total: 27.9s	remaining: 16.9s
622:	learn: 0.4158851	total: 27.9s	remaining: 16.9s
623:	learn: 0.4158720	total: 27.9s	remaining: 16.8s
624:	learn: 0.4158544	total: 28s	remaining: 16.8s
625:	learn: 0.4158360	total: 28s	remaining: 16.8s
626:	learn: 0.4158095	total: 28.1s	remaining: 16.7s
627:	learn: 0.4157963	total: 28.1s	remaining: 16.7s
628:	learn: 0.415778

773:	learn: 0.4130366	total: 34.8s	remaining: 10.2s
774:	learn: 0.4130215	total: 34.9s	remaining: 10.1s
775:	learn: 0.4130047	total: 34.9s	remaining: 10.1s
776:	learn: 0.4129872	total: 35s	remaining: 10s
777:	learn: 0.4129701	total: 35s	remaining: 9.98s
778:	learn: 0.4129493	total: 35s	remaining: 9.94s
779:	learn: 0.4129295	total: 35.1s	remaining: 9.89s
780:	learn: 0.4129130	total: 35.1s	remaining: 9.85s
781:	learn: 0.4128985	total: 35.1s	remaining: 9.8s
782:	learn: 0.4128811	total: 35.2s	remaining: 9.75s
783:	learn: 0.4128577	total: 35.2s	remaining: 9.7s
784:	learn: 0.4128427	total: 35.3s	remaining: 9.66s
785:	learn: 0.4128210	total: 35.3s	remaining: 9.62s
786:	learn: 0.4128094	total: 35.4s	remaining: 9.57s
787:	learn: 0.4127870	total: 35.4s	remaining: 9.53s
788:	learn: 0.4127667	total: 35.5s	remaining: 9.48s
789:	learn: 0.4127419	total: 35.5s	remaining: 9.44s
790:	learn: 0.4127300	total: 35.6s	remaining: 9.4s
791:	learn: 0.4127101	total: 35.6s	remaining: 9.35s
792:	learn: 0.4126881	t

932:	learn: 0.4102170	total: 42.2s	remaining: 3.03s
933:	learn: 0.4102053	total: 42.3s	remaining: 2.99s
934:	learn: 0.4101907	total: 42.3s	remaining: 2.94s
935:	learn: 0.4101718	total: 42.4s	remaining: 2.9s
936:	learn: 0.4101548	total: 42.4s	remaining: 2.85s
937:	learn: 0.4101402	total: 42.5s	remaining: 2.81s
938:	learn: 0.4101187	total: 42.5s	remaining: 2.76s
939:	learn: 0.4101019	total: 42.6s	remaining: 2.72s
940:	learn: 0.4100860	total: 42.6s	remaining: 2.67s
941:	learn: 0.4100696	total: 42.7s	remaining: 2.63s
942:	learn: 0.4100475	total: 42.7s	remaining: 2.58s
943:	learn: 0.4100375	total: 42.8s	remaining: 2.54s
944:	learn: 0.4100241	total: 42.8s	remaining: 2.49s
945:	learn: 0.4100020	total: 42.9s	remaining: 2.45s
946:	learn: 0.4099828	total: 42.9s	remaining: 2.4s
947:	learn: 0.4099652	total: 43s	remaining: 2.36s
948:	learn: 0.4099543	total: 43s	remaining: 2.31s
949:	learn: 0.4099380	total: 43s	remaining: 2.27s
950:	learn: 0.4099173	total: 43.1s	remaining: 2.22s
951:	learn: 0.409896

In [69]:
lgb.fit(x_dum.drop(['Age_scaled','Vintage_scaled','Avg_Account_Balance'], axis=1), y)
p = lgb.predict_proba(X_dum.drop(['Age_scaled','Vintage_scaled','Avg_Account_Balance'], axis=1))

In [70]:
p[:,1]

array([0.04028681, 0.31731056, 0.07317156, ..., 0.09124297, 0.45749094,
       0.07114944])

In [71]:
sub = pd.read_csv('sample_submission_eyYijxG.csv')



In [72]:
sub['Is_Lead'] = p[:,1]
sub.to_csv('Sub_3.csv', index=False)

# Approach 2
#### Feature Engineered Data

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(original, axis=1), y, test_size=0.13, shuffle=True)

data = [x_train, y_train, x_test, y_test]


In [39]:
model_names = []
model_performances = []

for model in models:
    perf = run_model(model, data)
    model_performances.append(perf)
    model_names.append(str(model.get_params).split('(')[0].split()[-1])
    print(model_names[-1], model_performances[-1],
          '\n\n------------------------------------------------------------------\n\n')
    

LogisticRegression 0.7586168036870014 

------------------------------------------------------------------


AdaBoostClassifier 0.7856953544128877 

------------------------------------------------------------------


BaggingClassifier 0.7311547728420779 

------------------------------------------------------------------


ExtraTreeClassifier 0.6144340250589423 

------------------------------------------------------------------


GradientBoostingClassifier 0.7945962764863197 

------------------------------------------------------------------


RandomForestClassifier 0.7409892175246728 

------------------------------------------------------------------


GaussianNB 0.7466182706464854 

------------------------------------------------------------------


BernoulliNB 0.7427003175181027 

------------------------------------------------------------------


KNeighborsClassifier 0.7233136428309087 

------------------------------------------------------------------


[17:19:17] WARNING: 

123:	learn: 0.4282979	total: 9.03s	remaining: 1m 3s
124:	learn: 0.4282723	total: 9.1s	remaining: 1m 3s
125:	learn: 0.4282191	total: 9.18s	remaining: 1m 3s
126:	learn: 0.4281909	total: 9.26s	remaining: 1m 3s
127:	learn: 0.4281495	total: 9.33s	remaining: 1m 3s
128:	learn: 0.4281249	total: 9.4s	remaining: 1m 3s
129:	learn: 0.4280548	total: 9.46s	remaining: 1m 3s
130:	learn: 0.4280074	total: 9.53s	remaining: 1m 3s
131:	learn: 0.4279633	total: 9.6s	remaining: 1m 3s
132:	learn: 0.4279243	total: 9.67s	remaining: 1m 3s
133:	learn: 0.4278769	total: 9.75s	remaining: 1m 3s
134:	learn: 0.4278484	total: 9.83s	remaining: 1m 2s
135:	learn: 0.4278272	total: 9.9s	remaining: 1m 2s
136:	learn: 0.4277975	total: 9.98s	remaining: 1m 2s
137:	learn: 0.4277600	total: 10.1s	remaining: 1m 2s
138:	learn: 0.4277139	total: 10.1s	remaining: 1m 2s
139:	learn: 0.4276786	total: 10.2s	remaining: 1m 2s
140:	learn: 0.4276495	total: 10.3s	remaining: 1m 2s
141:	learn: 0.4275929	total: 10.4s	remaining: 1m 2s
142:	learn: 0.42

284:	learn: 0.4234667	total: 21s	remaining: 52.8s
285:	learn: 0.4234541	total: 21.1s	remaining: 52.7s
286:	learn: 0.4234342	total: 21.2s	remaining: 52.6s
287:	learn: 0.4234022	total: 21.3s	remaining: 52.5s
288:	learn: 0.4233807	total: 21.3s	remaining: 52.5s
289:	learn: 0.4233604	total: 21.4s	remaining: 52.4s
290:	learn: 0.4232882	total: 21.5s	remaining: 52.3s
291:	learn: 0.4232558	total: 21.6s	remaining: 52.3s
292:	learn: 0.4232398	total: 21.6s	remaining: 52.2s
293:	learn: 0.4232181	total: 21.7s	remaining: 52.1s
294:	learn: 0.4231986	total: 21.8s	remaining: 52s
295:	learn: 0.4231712	total: 21.8s	remaining: 52s
296:	learn: 0.4231506	total: 21.9s	remaining: 51.9s
297:	learn: 0.4231193	total: 22s	remaining: 51.8s
298:	learn: 0.4230997	total: 22.1s	remaining: 51.8s
299:	learn: 0.4230674	total: 22.2s	remaining: 51.7s
300:	learn: 0.4230324	total: 22.2s	remaining: 51.6s
301:	learn: 0.4230070	total: 22.3s	remaining: 51.6s
302:	learn: 0.4229861	total: 22.4s	remaining: 51.5s
303:	learn: 0.422967

443:	learn: 0.4199016	total: 32.9s	remaining: 41.2s
444:	learn: 0.4198775	total: 33s	remaining: 41.2s
445:	learn: 0.4198517	total: 33.1s	remaining: 41.1s
446:	learn: 0.4198324	total: 33.2s	remaining: 41s
447:	learn: 0.4198184	total: 33.2s	remaining: 40.9s
448:	learn: 0.4197956	total: 33.3s	remaining: 40.9s
449:	learn: 0.4197817	total: 33.4s	remaining: 40.8s
450:	learn: 0.4197608	total: 33.5s	remaining: 40.7s
451:	learn: 0.4197403	total: 33.5s	remaining: 40.6s
452:	learn: 0.4197189	total: 33.6s	remaining: 40.6s
453:	learn: 0.4196982	total: 33.7s	remaining: 40.5s
454:	learn: 0.4196775	total: 33.8s	remaining: 40.4s
455:	learn: 0.4196538	total: 33.8s	remaining: 40.4s
456:	learn: 0.4196317	total: 33.9s	remaining: 40.3s
457:	learn: 0.4196116	total: 34s	remaining: 40.2s
458:	learn: 0.4195897	total: 34.1s	remaining: 40.1s
459:	learn: 0.4195714	total: 34.1s	remaining: 40.1s
460:	learn: 0.4195499	total: 34.2s	remaining: 40s
461:	learn: 0.4195330	total: 34.3s	remaining: 39.9s
462:	learn: 0.419514

602:	learn: 0.4170606	total: 44.8s	remaining: 29.5s
603:	learn: 0.4170449	total: 44.8s	remaining: 29.4s
604:	learn: 0.4170301	total: 44.9s	remaining: 29.3s
605:	learn: 0.4170059	total: 45s	remaining: 29.2s
606:	learn: 0.4169842	total: 45s	remaining: 29.2s
607:	learn: 0.4169639	total: 45.1s	remaining: 29.1s
608:	learn: 0.4169507	total: 45.2s	remaining: 29s
609:	learn: 0.4169357	total: 45.3s	remaining: 28.9s
610:	learn: 0.4169176	total: 45.3s	remaining: 28.9s
611:	learn: 0.4168969	total: 45.4s	remaining: 28.8s
612:	learn: 0.4168732	total: 45.5s	remaining: 28.7s
613:	learn: 0.4168476	total: 45.6s	remaining: 28.6s
614:	learn: 0.4168402	total: 45.6s	remaining: 28.6s
615:	learn: 0.4168268	total: 45.7s	remaining: 28.5s
616:	learn: 0.4168142	total: 45.8s	remaining: 28.4s
617:	learn: 0.4168009	total: 45.9s	remaining: 28.4s
618:	learn: 0.4167908	total: 45.9s	remaining: 28.3s
619:	learn: 0.4167755	total: 46s	remaining: 28.2s
620:	learn: 0.4167581	total: 46.1s	remaining: 28.1s
621:	learn: 0.416738

761:	learn: 0.4143236	total: 56.6s	remaining: 17.7s
762:	learn: 0.4143079	total: 56.7s	remaining: 17.6s
763:	learn: 0.4142956	total: 56.8s	remaining: 17.5s
764:	learn: 0.4142772	total: 56.8s	remaining: 17.5s
765:	learn: 0.4142648	total: 56.9s	remaining: 17.4s
766:	learn: 0.4142483	total: 57s	remaining: 17.3s
767:	learn: 0.4142357	total: 57.1s	remaining: 17.2s
768:	learn: 0.4142217	total: 57.1s	remaining: 17.2s
769:	learn: 0.4142017	total: 57.2s	remaining: 17.1s
770:	learn: 0.4141869	total: 57.3s	remaining: 17s
771:	learn: 0.4141715	total: 57.4s	remaining: 16.9s
772:	learn: 0.4141577	total: 57.4s	remaining: 16.9s
773:	learn: 0.4141430	total: 57.5s	remaining: 16.8s
774:	learn: 0.4141223	total: 57.6s	remaining: 16.7s
775:	learn: 0.4141036	total: 57.7s	remaining: 16.6s
776:	learn: 0.4140892	total: 57.7s	remaining: 16.6s
777:	learn: 0.4140777	total: 57.8s	remaining: 16.5s
778:	learn: 0.4140566	total: 57.9s	remaining: 16.4s
779:	learn: 0.4140353	total: 58s	remaining: 16.3s
780:	learn: 0.4140

922:	learn: 0.4116799	total: 1m 8s	remaining: 5.73s
923:	learn: 0.4116659	total: 1m 8s	remaining: 5.66s
924:	learn: 0.4116484	total: 1m 8s	remaining: 5.58s
925:	learn: 0.4116179	total: 1m 8s	remaining: 5.51s
926:	learn: 0.4116068	total: 1m 8s	remaining: 5.43s
927:	learn: 0.4115917	total: 1m 9s	remaining: 5.36s
928:	learn: 0.4115712	total: 1m 9s	remaining: 5.28s
929:	learn: 0.4115505	total: 1m 9s	remaining: 5.21s
930:	learn: 0.4115396	total: 1m 9s	remaining: 5.13s
931:	learn: 0.4115184	total: 1m 9s	remaining: 5.06s
932:	learn: 0.4115077	total: 1m 9s	remaining: 4.99s
933:	learn: 0.4114881	total: 1m 9s	remaining: 4.91s
934:	learn: 0.4114656	total: 1m 9s	remaining: 4.84s
935:	learn: 0.4114465	total: 1m 9s	remaining: 4.76s
936:	learn: 0.4114341	total: 1m 9s	remaining: 4.69s
937:	learn: 0.4114242	total: 1m 9s	remaining: 4.61s
938:	learn: 0.4114096	total: 1m 9s	remaining: 4.54s
939:	learn: 0.4113875	total: 1m 9s	remaining: 4.46s
940:	learn: 0.4113815	total: 1m 10s	remaining: 4.39s
941:	learn:

# Approach 3
#### Cat Feats

In [106]:
x

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Avg_Account_Balance,Is_Active,Credit_Product,Avg_Account_Balance_Unskewed,Age_scaled,Vintage_scaled
0,Female,73,RG268,Other,X3,43,1045696,No,No,13.860194,1.963311,-0.121384
1,Female,30,RG277,Salaried,X1,32,581988,No,No,13.274207,-0.933890,-0.461633
2,Female,56,RG268,Self_Employed,X3,26,1484315,Yes,No,14.210465,0.817906,-0.647223
3,Male,34,RG270,Salaried,X1,19,470454,No,No,13.061456,-0.664383,-0.863745
4,Female,30,RG282,Salaried,X1,33,886787,No,No,13.695361,-0.933890,-0.430701
...,...,...,...,...,...,...,...,...,...,...,...,...
245720,Male,51,RG284,Self_Employed,X3,109,1925586,No,Yes,14.470741,0.481022,1.920111
245721,Male,27,RG268,Salaried,X1,15,862952,Yes,No,13.668116,-1.136021,-0.987472
245722,Female,26,RG281,Salaried,X1,13,670659,No,No,13.416018,-1.203397,-1.049336
245723,Female,28,RG273,Salaried,X1,31,407504,No,No,12.917808,-1.068644,-0.492564


In [44]:
cat_feats = ['Gender_encoded','Region_Code_encoded','Occupation_encoded','Channel_Code_encoded','Is_Active_encoded','Credit_Product_encoded']

### Original data first

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(fe, axis=1), y, test_size=0.13, shuffle=True)

data = [x_train, y_train, x_test, y_test]

catb = CatBoostClassifier(cat_features=cat_feats)
run_model(catb, data)

Learning rate set to 0.101819
0:	learn: 0.6348750	total: 339ms	remaining: 5m 39s
1:	learn: 0.5898382	total: 668ms	remaining: 5m 33s
2:	learn: 0.5568081	total: 946ms	remaining: 5m 14s
3:	learn: 0.5314716	total: 1.25s	remaining: 5m 12s
4:	learn: 0.5121097	total: 1.52s	remaining: 5m 3s
5:	learn: 0.4967315	total: 1.81s	remaining: 4m 59s
6:	learn: 0.4851224	total: 2.08s	remaining: 4m 55s
7:	learn: 0.4757318	total: 2.34s	remaining: 4m 50s
8:	learn: 0.4687281	total: 2.63s	remaining: 4m 49s
9:	learn: 0.4621943	total: 2.93s	remaining: 4m 50s
10:	learn: 0.4574895	total: 3.19s	remaining: 4m 47s
11:	learn: 0.4539261	total: 3.42s	remaining: 4m 42s
12:	learn: 0.4508784	total: 3.69s	remaining: 4m 40s
13:	learn: 0.4485360	total: 3.98s	remaining: 4m 40s
14:	learn: 0.4465790	total: 4.26s	remaining: 4m 39s
15:	learn: 0.4447758	total: 4.52s	remaining: 4m 37s
16:	learn: 0.4433718	total: 4.78s	remaining: 4m 36s
17:	learn: 0.4420708	total: 5.03s	remaining: 4m 34s
18:	learn: 0.4411094	total: 5.31s	remaining: 

157:	learn: 0.4275901	total: 40.8s	remaining: 3m 37s
158:	learn: 0.4275608	total: 41.1s	remaining: 3m 37s
159:	learn: 0.4275008	total: 41.4s	remaining: 3m 37s
160:	learn: 0.4274853	total: 41.7s	remaining: 3m 37s
161:	learn: 0.4274385	total: 42s	remaining: 3m 37s
162:	learn: 0.4273859	total: 42.3s	remaining: 3m 37s
163:	learn: 0.4273787	total: 42.6s	remaining: 3m 37s
164:	learn: 0.4273705	total: 42.9s	remaining: 3m 36s
165:	learn: 0.4273438	total: 43.2s	remaining: 3m 36s
166:	learn: 0.4273191	total: 43.4s	remaining: 3m 36s
167:	learn: 0.4272863	total: 43.7s	remaining: 3m 36s
168:	learn: 0.4272285	total: 44s	remaining: 3m 36s
169:	learn: 0.4272195	total: 44.3s	remaining: 3m 36s
170:	learn: 0.4272061	total: 44.6s	remaining: 3m 36s
171:	learn: 0.4271856	total: 44.9s	remaining: 3m 36s
172:	learn: 0.4271227	total: 45.2s	remaining: 3m 36s
173:	learn: 0.4271111	total: 45.5s	remaining: 3m 35s
174:	learn: 0.4271063	total: 45.7s	remaining: 3m 35s
175:	learn: 0.4271034	total: 45.9s	remaining: 3m 3

312:	learn: 0.4246000	total: 1m 21s	remaining: 2m 58s
313:	learn: 0.4245833	total: 1m 21s	remaining: 2m 58s
314:	learn: 0.4245759	total: 1m 21s	remaining: 2m 57s
315:	learn: 0.4245553	total: 1m 22s	remaining: 2m 57s
316:	learn: 0.4245469	total: 1m 22s	remaining: 2m 57s
317:	learn: 0.4245301	total: 1m 22s	remaining: 2m 57s
318:	learn: 0.4245190	total: 1m 22s	remaining: 2m 56s
319:	learn: 0.4244983	total: 1m 22s	remaining: 2m 56s
320:	learn: 0.4244933	total: 1m 23s	remaining: 2m 55s
321:	learn: 0.4244892	total: 1m 23s	remaining: 2m 55s
322:	learn: 0.4244784	total: 1m 23s	remaining: 2m 55s
323:	learn: 0.4244522	total: 1m 23s	remaining: 2m 54s
324:	learn: 0.4244436	total: 1m 23s	remaining: 2m 54s
325:	learn: 0.4244355	total: 1m 24s	remaining: 2m 53s
326:	learn: 0.4244334	total: 1m 24s	remaining: 2m 53s
327:	learn: 0.4244257	total: 1m 24s	remaining: 2m 53s
328:	learn: 0.4244032	total: 1m 24s	remaining: 2m 52s
329:	learn: 0.4243948	total: 1m 24s	remaining: 2m 52s
330:	learn: 0.4243826	total:

464:	learn: 0.4220991	total: 1m 56s	remaining: 2m 13s
465:	learn: 0.4220864	total: 1m 56s	remaining: 2m 13s
466:	learn: 0.4220823	total: 1m 56s	remaining: 2m 13s
467:	learn: 0.4220739	total: 1m 56s	remaining: 2m 12s
468:	learn: 0.4220624	total: 1m 56s	remaining: 2m 12s
469:	learn: 0.4220446	total: 1m 57s	remaining: 2m 12s
470:	learn: 0.4220238	total: 1m 57s	remaining: 2m 11s
471:	learn: 0.4220044	total: 1m 57s	remaining: 2m 11s
472:	learn: 0.4219764	total: 1m 57s	remaining: 2m 11s
473:	learn: 0.4219588	total: 1m 58s	remaining: 2m 11s
474:	learn: 0.4219421	total: 1m 58s	remaining: 2m 10s
475:	learn: 0.4219381	total: 1m 58s	remaining: 2m 10s
476:	learn: 0.4219346	total: 1m 58s	remaining: 2m 10s
477:	learn: 0.4219286	total: 1m 58s	remaining: 2m 9s
478:	learn: 0.4219164	total: 1m 59s	remaining: 2m 9s
479:	learn: 0.4219144	total: 1m 59s	remaining: 2m 9s
480:	learn: 0.4219069	total: 1m 59s	remaining: 2m 9s
481:	learn: 0.4218989	total: 1m 59s	remaining: 2m 8s
482:	learn: 0.4218882	total: 2m	r

618:	learn: 0.4199932	total: 2m 25s	remaining: 1m 29s
619:	learn: 0.4199817	total: 2m 26s	remaining: 1m 29s
620:	learn: 0.4199697	total: 2m 26s	remaining: 1m 29s
621:	learn: 0.4199534	total: 2m 26s	remaining: 1m 28s
622:	learn: 0.4199402	total: 2m 26s	remaining: 1m 28s
623:	learn: 0.4199300	total: 2m 26s	remaining: 1m 28s
624:	learn: 0.4199178	total: 2m 26s	remaining: 1m 27s
625:	learn: 0.4199093	total: 2m 26s	remaining: 1m 27s
626:	learn: 0.4198963	total: 2m 26s	remaining: 1m 27s
627:	learn: 0.4198844	total: 2m 27s	remaining: 1m 27s
628:	learn: 0.4198650	total: 2m 27s	remaining: 1m 26s
629:	learn: 0.4198503	total: 2m 27s	remaining: 1m 26s
630:	learn: 0.4198376	total: 2m 27s	remaining: 1m 26s
631:	learn: 0.4198198	total: 2m 27s	remaining: 1m 25s
632:	learn: 0.4198007	total: 2m 27s	remaining: 1m 25s
633:	learn: 0.4197883	total: 2m 27s	remaining: 1m 25s
634:	learn: 0.4197699	total: 2m 27s	remaining: 1m 25s
635:	learn: 0.4197634	total: 2m 28s	remaining: 1m 24s
636:	learn: 0.4197538	total:

773:	learn: 0.4178622	total: 2m 45s	remaining: 48.4s
774:	learn: 0.4178569	total: 2m 45s	remaining: 48.2s
775:	learn: 0.4178422	total: 2m 46s	remaining: 47.9s
776:	learn: 0.4178231	total: 2m 46s	remaining: 47.7s
777:	learn: 0.4178164	total: 2m 46s	remaining: 47.5s
778:	learn: 0.4178007	total: 2m 46s	remaining: 47.2s
779:	learn: 0.4177892	total: 2m 46s	remaining: 47s
780:	learn: 0.4177809	total: 2m 46s	remaining: 46.8s
781:	learn: 0.4177724	total: 2m 46s	remaining: 46.5s
782:	learn: 0.4177524	total: 2m 46s	remaining: 46.3s
783:	learn: 0.4177425	total: 2m 47s	remaining: 46s
784:	learn: 0.4177333	total: 2m 47s	remaining: 45.8s
785:	learn: 0.4177254	total: 2m 47s	remaining: 45.6s
786:	learn: 0.4177056	total: 2m 47s	remaining: 45.3s
787:	learn: 0.4176791	total: 2m 47s	remaining: 45.1s
788:	learn: 0.4176665	total: 2m 47s	remaining: 44.9s
789:	learn: 0.4176544	total: 2m 47s	remaining: 44.6s
790:	learn: 0.4176503	total: 2m 48s	remaining: 44.4s
791:	learn: 0.4176442	total: 2m 48s	remaining: 44.

930:	learn: 0.4160029	total: 3m 5s	remaining: 13.8s
931:	learn: 0.4159839	total: 3m 5s	remaining: 13.6s
932:	learn: 0.4159559	total: 3m 6s	remaining: 13.4s
933:	learn: 0.4159290	total: 3m 6s	remaining: 13.2s
934:	learn: 0.4159083	total: 3m 6s	remaining: 13s
935:	learn: 0.4158904	total: 3m 6s	remaining: 12.8s
936:	learn: 0.4158765	total: 3m 6s	remaining: 12.5s
937:	learn: 0.4158634	total: 3m 6s	remaining: 12.3s
938:	learn: 0.4158540	total: 3m 6s	remaining: 12.1s
939:	learn: 0.4158482	total: 3m 6s	remaining: 11.9s
940:	learn: 0.4158358	total: 3m 7s	remaining: 11.7s
941:	learn: 0.4158139	total: 3m 7s	remaining: 11.5s
942:	learn: 0.4158065	total: 3m 7s	remaining: 11.3s
943:	learn: 0.4157978	total: 3m 7s	remaining: 11.1s
944:	learn: 0.4157836	total: 3m 7s	remaining: 10.9s
945:	learn: 0.4157730	total: 3m 7s	remaining: 10.7s
946:	learn: 0.4157639	total: 3m 7s	remaining: 10.5s
947:	learn: 0.4157554	total: 3m 8s	remaining: 10.3s
948:	learn: 0.4157372	total: 3m 8s	remaining: 10.1s
949:	learn: 0.

0.8007292398490617

## FE data

In [46]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(original, axis=1), y, test_size=0.13, shuffle=True)

data = [x_train, y_train, x_test, y_test]

catb = CatBoostClassifier(cat_features=cat_feats)
run_model(catb, data)

Learning rate set to 0.101819
0:	learn: 0.6354271	total: 116ms	remaining: 1m 55s
1:	learn: 0.5914095	total: 237ms	remaining: 1m 58s
2:	learn: 0.5566101	total: 363ms	remaining: 2m
3:	learn: 0.5312065	total: 482ms	remaining: 2m
4:	learn: 0.5114962	total: 598ms	remaining: 1m 59s
5:	learn: 0.4958171	total: 711ms	remaining: 1m 57s
6:	learn: 0.4844561	total: 827ms	remaining: 1m 57s
7:	learn: 0.4749284	total: 959ms	remaining: 1m 58s
8:	learn: 0.4677704	total: 1.08s	remaining: 1m 58s
9:	learn: 0.4618153	total: 1.2s	remaining: 1m 58s
10:	learn: 0.4573678	total: 1.32s	remaining: 1m 58s
11:	learn: 0.4538303	total: 1.41s	remaining: 1m 55s
12:	learn: 0.4507665	total: 1.51s	remaining: 1m 54s
13:	learn: 0.4479731	total: 1.63s	remaining: 1m 54s
14:	learn: 0.4459447	total: 1.77s	remaining: 1m 56s
15:	learn: 0.4443644	total: 1.91s	remaining: 1m 57s
16:	learn: 0.4429535	total: 2.02s	remaining: 1m 56s
17:	learn: 0.4417192	total: 2.12s	remaining: 1m 55s
18:	learn: 0.4406451	total: 2.23s	remaining: 1m 55s
1

157:	learn: 0.4278697	total: 18.8s	remaining: 1m 40s
158:	learn: 0.4278345	total: 18.9s	remaining: 1m 40s
159:	learn: 0.4278028	total: 19.1s	remaining: 1m 40s
160:	learn: 0.4277661	total: 19.2s	remaining: 1m 39s
161:	learn: 0.4277360	total: 19.3s	remaining: 1m 39s
162:	learn: 0.4277121	total: 19.4s	remaining: 1m 39s
163:	learn: 0.4276757	total: 19.5s	remaining: 1m 39s
164:	learn: 0.4276428	total: 19.6s	remaining: 1m 39s
165:	learn: 0.4276130	total: 19.8s	remaining: 1m 39s
166:	learn: 0.4275948	total: 19.9s	remaining: 1m 39s
167:	learn: 0.4275624	total: 20s	remaining: 1m 39s
168:	learn: 0.4275477	total: 20.1s	remaining: 1m 38s
169:	learn: 0.4275114	total: 20.2s	remaining: 1m 38s
170:	learn: 0.4275087	total: 20.3s	remaining: 1m 38s
171:	learn: 0.4274750	total: 20.4s	remaining: 1m 38s
172:	learn: 0.4274584	total: 20.6s	remaining: 1m 38s
173:	learn: 0.4274291	total: 20.7s	remaining: 1m 38s
174:	learn: 0.4273706	total: 20.9s	remaining: 1m 38s
175:	learn: 0.4273573	total: 21s	remaining: 1m 3

313:	learn: 0.4244879	total: 38.2s	remaining: 1m 23s
314:	learn: 0.4244726	total: 38.4s	remaining: 1m 23s
315:	learn: 0.4244457	total: 38.5s	remaining: 1m 23s
316:	learn: 0.4244001	total: 38.6s	remaining: 1m 23s
317:	learn: 0.4243951	total: 38.8s	remaining: 1m 23s
318:	learn: 0.4243910	total: 38.9s	remaining: 1m 23s
319:	learn: 0.4243843	total: 39.1s	remaining: 1m 22s
320:	learn: 0.4243541	total: 39.2s	remaining: 1m 22s
321:	learn: 0.4243523	total: 39.3s	remaining: 1m 22s
322:	learn: 0.4243272	total: 39.4s	remaining: 1m 22s
323:	learn: 0.4243122	total: 39.6s	remaining: 1m 22s
324:	learn: 0.4242921	total: 39.7s	remaining: 1m 22s
325:	learn: 0.4242733	total: 39.8s	remaining: 1m 22s
326:	learn: 0.4242569	total: 40s	remaining: 1m 22s
327:	learn: 0.4242447	total: 40.1s	remaining: 1m 22s
328:	learn: 0.4242262	total: 40.2s	remaining: 1m 22s
329:	learn: 0.4242077	total: 40.4s	remaining: 1m 21s
330:	learn: 0.4242017	total: 40.5s	remaining: 1m 21s
331:	learn: 0.4241928	total: 40.6s	remaining: 1m

469:	learn: 0.4221385	total: 58s	remaining: 1m 5s
470:	learn: 0.4221249	total: 58.1s	remaining: 1m 5s
471:	learn: 0.4221189	total: 58.2s	remaining: 1m 5s
472:	learn: 0.4220981	total: 58.4s	remaining: 1m 5s
473:	learn: 0.4220862	total: 58.5s	remaining: 1m 4s
474:	learn: 0.4220775	total: 58.6s	remaining: 1m 4s
475:	learn: 0.4220744	total: 58.7s	remaining: 1m 4s
476:	learn: 0.4220512	total: 58.9s	remaining: 1m 4s
477:	learn: 0.4220302	total: 59s	remaining: 1m 4s
478:	learn: 0.4220060	total: 59.1s	remaining: 1m 4s
479:	learn: 0.4219914	total: 59.3s	remaining: 1m 4s
480:	learn: 0.4219844	total: 59.4s	remaining: 1m 4s
481:	learn: 0.4219796	total: 59.5s	remaining: 1m 3s
482:	learn: 0.4219707	total: 59.6s	remaining: 1m 3s
483:	learn: 0.4219594	total: 59.7s	remaining: 1m 3s
484:	learn: 0.4219506	total: 59.8s	remaining: 1m 3s
485:	learn: 0.4219310	total: 60s	remaining: 1m 3s
486:	learn: 0.4219156	total: 1m	remaining: 1m 3s
487:	learn: 0.4219023	total: 1m	remaining: 1m 3s
488:	learn: 0.4218923	to

627:	learn: 0.4200504	total: 1m 18s	remaining: 46.3s
628:	learn: 0.4200460	total: 1m 18s	remaining: 46.2s
629:	learn: 0.4200440	total: 1m 18s	remaining: 46.1s
630:	learn: 0.4200351	total: 1m 18s	remaining: 45.9s
631:	learn: 0.4200232	total: 1m 18s	remaining: 45.8s
632:	learn: 0.4200152	total: 1m 18s	remaining: 45.7s
633:	learn: 0.4200042	total: 1m 18s	remaining: 45.6s
634:	learn: 0.4200008	total: 1m 19s	remaining: 45.5s
635:	learn: 0.4200000	total: 1m 19s	remaining: 45.4s
636:	learn: 0.4199683	total: 1m 19s	remaining: 45.2s
637:	learn: 0.4199517	total: 1m 19s	remaining: 45.1s
638:	learn: 0.4199309	total: 1m 19s	remaining: 45s
639:	learn: 0.4199092	total: 1m 19s	remaining: 44.9s
640:	learn: 0.4198829	total: 1m 19s	remaining: 44.7s
641:	learn: 0.4198702	total: 1m 20s	remaining: 44.6s
642:	learn: 0.4198488	total: 1m 20s	remaining: 44.5s
643:	learn: 0.4198365	total: 1m 20s	remaining: 44.4s
644:	learn: 0.4198255	total: 1m 20s	remaining: 44.3s
645:	learn: 0.4198159	total: 1m 20s	remaining: 4

783:	learn: 0.4179031	total: 1m 38s	remaining: 27.1s
784:	learn: 0.4178954	total: 1m 38s	remaining: 27s
785:	learn: 0.4178729	total: 1m 38s	remaining: 26.8s
786:	learn: 0.4178662	total: 1m 38s	remaining: 26.7s
787:	learn: 0.4178592	total: 1m 38s	remaining: 26.6s
788:	learn: 0.4178465	total: 1m 38s	remaining: 26.5s
789:	learn: 0.4178422	total: 1m 39s	remaining: 26.3s
790:	learn: 0.4178286	total: 1m 39s	remaining: 26.2s
791:	learn: 0.4178140	total: 1m 39s	remaining: 26.1s
792:	learn: 0.4178091	total: 1m 39s	remaining: 26s
793:	learn: 0.4177924	total: 1m 39s	remaining: 25.8s
794:	learn: 0.4177852	total: 1m 39s	remaining: 25.7s
795:	learn: 0.4177764	total: 1m 39s	remaining: 25.6s
796:	learn: 0.4177654	total: 1m 39s	remaining: 25.5s
797:	learn: 0.4177510	total: 1m 40s	remaining: 25.3s
798:	learn: 0.4177322	total: 1m 40s	remaining: 25.2s
799:	learn: 0.4177218	total: 1m 40s	remaining: 25.1s
800:	learn: 0.4177064	total: 1m 40s	remaining: 25s
801:	learn: 0.4176983	total: 1m 40s	remaining: 24.9s

940:	learn: 0.4159738	total: 1m 59s	remaining: 7.47s
941:	learn: 0.4159515	total: 1m 59s	remaining: 7.35s
942:	learn: 0.4159489	total: 1m 59s	remaining: 7.22s
943:	learn: 0.4159436	total: 1m 59s	remaining: 7.09s
944:	learn: 0.4159328	total: 1m 59s	remaining: 6.97s
945:	learn: 0.4159266	total: 1m 59s	remaining: 6.85s
946:	learn: 0.4159081	total: 2m	remaining: 6.72s
947:	learn: 0.4158890	total: 2m	remaining: 6.6s
948:	learn: 0.4158791	total: 2m	remaining: 6.47s
949:	learn: 0.4158609	total: 2m	remaining: 6.35s
950:	learn: 0.4158516	total: 2m	remaining: 6.22s
951:	learn: 0.4158458	total: 2m	remaining: 6.1s
952:	learn: 0.4158317	total: 2m 1s	remaining: 5.97s
953:	learn: 0.4158114	total: 2m 1s	remaining: 5.85s
954:	learn: 0.4157977	total: 2m 1s	remaining: 5.72s
955:	learn: 0.4157903	total: 2m 1s	remaining: 5.6s
956:	learn: 0.4157727	total: 2m 1s	remaining: 5.47s
957:	learn: 0.4157629	total: 2m 1s	remaining: 5.35s
958:	learn: 0.4157452	total: 2m 2s	remaining: 5.22s
959:	learn: 0.4157305	total

0.8019684443979558

# Approach 4

### Stacking

In [73]:
x_train, x_test, y_train, y_test = train_test_split(x_dum.drop(original, axis=1), y, test_size=0.13, shuffle=True)


In [74]:


stack = StackingCVClassifier(classifiers=models, meta_classifier=lgb,
                            cv=10, use_features_in_secondary=True, 
                            store_train_meta_features=True, shuffle=False)


In [75]:
stack.fit(x_train, y_train)

[21:45:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:45:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:46:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:46:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

39:	learn: 0.4339234	total: 1.82s	remaining: 43.7s
40:	learn: 0.4337982	total: 1.86s	remaining: 43.6s
41:	learn: 0.4336906	total: 1.9s	remaining: 43.4s
42:	learn: 0.4335646	total: 1.94s	remaining: 43.2s
43:	learn: 0.4334457	total: 1.98s	remaining: 43s
44:	learn: 0.4332617	total: 2.03s	remaining: 43s
45:	learn: 0.4332018	total: 2.07s	remaining: 43s
46:	learn: 0.4331369	total: 2.12s	remaining: 42.9s
47:	learn: 0.4330343	total: 2.16s	remaining: 42.8s
48:	learn: 0.4329616	total: 2.21s	remaining: 42.8s
49:	learn: 0.4328820	total: 2.25s	remaining: 42.7s
50:	learn: 0.4328371	total: 2.29s	remaining: 42.6s
51:	learn: 0.4327013	total: 2.33s	remaining: 42.6s
52:	learn: 0.4326454	total: 2.38s	remaining: 42.5s
53:	learn: 0.4324210	total: 2.42s	remaining: 42.4s
54:	learn: 0.4323483	total: 2.48s	remaining: 42.7s
55:	learn: 0.4323016	total: 2.53s	remaining: 42.6s
56:	learn: 0.4321362	total: 2.56s	remaining: 42.4s
57:	learn: 0.4320663	total: 2.63s	remaining: 42.6s
58:	learn: 0.4320154	total: 2.67s	rema

199:	learn: 0.4259556	total: 9.33s	remaining: 37.3s
200:	learn: 0.4259241	total: 9.38s	remaining: 37.3s
201:	learn: 0.4258941	total: 9.42s	remaining: 37.2s
202:	learn: 0.4258631	total: 9.47s	remaining: 37.2s
203:	learn: 0.4258361	total: 9.53s	remaining: 37.2s
204:	learn: 0.4258166	total: 9.57s	remaining: 37.1s
205:	learn: 0.4257821	total: 9.62s	remaining: 37.1s
206:	learn: 0.4257388	total: 9.67s	remaining: 37s
207:	learn: 0.4257111	total: 9.72s	remaining: 37s
208:	learn: 0.4256919	total: 9.78s	remaining: 37s
209:	learn: 0.4256623	total: 9.84s	remaining: 37s
210:	learn: 0.4256294	total: 9.88s	remaining: 37s
211:	learn: 0.4255944	total: 9.94s	remaining: 36.9s
212:	learn: 0.4255687	total: 9.98s	remaining: 36.9s
213:	learn: 0.4255433	total: 10s	remaining: 36.8s
214:	learn: 0.4255106	total: 10.1s	remaining: 36.8s
215:	learn: 0.4254735	total: 10.1s	remaining: 36.8s
216:	learn: 0.4254452	total: 10.2s	remaining: 36.7s
217:	learn: 0.4254124	total: 10.2s	remaining: 36.7s
218:	learn: 0.4253844	to

361:	learn: 0.4212799	total: 17.1s	remaining: 30.1s
362:	learn: 0.4212506	total: 17.1s	remaining: 30s
363:	learn: 0.4212238	total: 17.2s	remaining: 30s
364:	learn: 0.4211968	total: 17.2s	remaining: 29.9s
365:	learn: 0.4211683	total: 17.3s	remaining: 29.9s
366:	learn: 0.4211448	total: 17.3s	remaining: 29.8s
367:	learn: 0.4211279	total: 17.3s	remaining: 29.8s
368:	learn: 0.4211007	total: 17.4s	remaining: 29.7s
369:	learn: 0.4210705	total: 17.4s	remaining: 29.7s
370:	learn: 0.4210498	total: 17.5s	remaining: 29.7s
371:	learn: 0.4210347	total: 17.5s	remaining: 29.6s
372:	learn: 0.4210053	total: 17.6s	remaining: 29.6s
373:	learn: 0.4209871	total: 17.6s	remaining: 29.5s
374:	learn: 0.4209536	total: 17.7s	remaining: 29.5s
375:	learn: 0.4209343	total: 17.7s	remaining: 29.4s
376:	learn: 0.4209187	total: 17.8s	remaining: 29.4s
377:	learn: 0.4208984	total: 17.8s	remaining: 29.3s
378:	learn: 0.4208797	total: 17.9s	remaining: 29.3s
379:	learn: 0.4208602	total: 17.9s	remaining: 29.2s
380:	learn: 0.42

522:	learn: 0.4174991	total: 24.8s	remaining: 22.6s
523:	learn: 0.4174792	total: 24.8s	remaining: 22.5s
524:	learn: 0.4174591	total: 24.8s	remaining: 22.5s
525:	learn: 0.4174376	total: 24.9s	remaining: 22.4s
526:	learn: 0.4174193	total: 24.9s	remaining: 22.4s
527:	learn: 0.4173997	total: 25s	remaining: 22.4s
528:	learn: 0.4173779	total: 25s	remaining: 22.3s
529:	learn: 0.4173543	total: 25.1s	remaining: 22.3s
530:	learn: 0.4173343	total: 25.1s	remaining: 22.2s
531:	learn: 0.4173165	total: 25.2s	remaining: 22.2s
532:	learn: 0.4173043	total: 25.3s	remaining: 22.1s
533:	learn: 0.4172754	total: 25.3s	remaining: 22.1s
534:	learn: 0.4172595	total: 25.3s	remaining: 22s
535:	learn: 0.4172347	total: 25.4s	remaining: 22s
536:	learn: 0.4172035	total: 25.4s	remaining: 21.9s
537:	learn: 0.4171755	total: 25.5s	remaining: 21.9s
538:	learn: 0.4171587	total: 25.5s	remaining: 21.8s
539:	learn: 0.4171354	total: 25.6s	remaining: 21.8s
540:	learn: 0.4171080	total: 25.6s	remaining: 21.7s
541:	learn: 0.417081

...

489:	learn: 0.4190605	total: 25.7s	remaining: 26.8s
490:	learn: 0.4190325	total: 25.8s	remaining: 26.7s
491:	learn: 0.4190169	total: 25.8s	remaining: 26.7s
492:	learn: 0.4189944	total: 25.9s	remaining: 26.6s
493:	learn: 0.4189737	total: 25.9s	remaining: 26.6s
494:	learn: 0.4189520	total: 26s	remaining: 26.5s
495:	learn: 0.4189301	total: 26s	remaining: 26.4s
496:	learn: 0.4189108	total: 26.1s	remaining: 26.4s
497:	learn: 0.4188839	total: 26.1s	remaining: 26.3s
498:	learn: 0.4188667	total: 26.2s	remaining: 26.3s
499:	learn: 0.4188452	total: 26.2s	remaining: 26.2s
500:	learn: 0.4188265	total: 26.3s	remaining: 26.2s
501:	learn: 0.4188031	total: 26.3s	remaining: 26.1s
502:	learn: 0.4187817	total: 26.4s	remaining: 26.1s
503:	learn: 0.4187607	total: 26.5s	remaining: 26s
504:	learn: 0.4187374	total: 26.5s	remaining: 26s
505:	learn: 0.4187204	total: 26.6s	remaining: 25.9s
506:	learn: 0.4187061	total: 26.6s	remaining: 25.9s
507:	learn: 0.4186848	total: 26.7s	remaining: 25.8s
508:	learn: 0.418666

648:	learn: 0.4159134	total: 34.2s	remaining: 18.5s
649:	learn: 0.4159040	total: 34.2s	remaining: 18.4s
650:	learn: 0.4158896	total: 34.3s	remaining: 18.4s
651:	learn: 0.4158693	total: 34.3s	remaining: 18.3s
652:	learn: 0.4158504	total: 34.4s	remaining: 18.3s
653:	learn: 0.4158352	total: 34.5s	remaining: 18.2s
654:	learn: 0.4158182	total: 34.5s	remaining: 18.2s
655:	learn: 0.4158017	total: 34.6s	remaining: 18.1s
656:	learn: 0.4157816	total: 34.6s	remaining: 18.1s
657:	learn: 0.4157582	total: 34.6s	remaining: 18s
658:	learn: 0.4157388	total: 34.7s	remaining: 18s
659:	learn: 0.4157154	total: 34.8s	remaining: 17.9s
660:	learn: 0.4156908	total: 34.8s	remaining: 17.9s
661:	learn: 0.4156695	total: 34.9s	remaining: 17.8s
662:	learn: 0.4156543	total: 34.9s	remaining: 17.7s
663:	learn: 0.4156359	total: 35s	remaining: 17.7s
664:	learn: 0.4156187	total: 35s	remaining: 17.6s
665:	learn: 0.4155971	total: 35.1s	remaining: 17.6s
666:	learn: 0.4155820	total: 35.1s	remaining: 17.5s
667:	learn: 0.415561

808:	learn: 0.4128823	total: 42.9s	remaining: 10.1s
809:	learn: 0.4128641	total: 43s	remaining: 10.1s
810:	learn: 0.4128464	total: 43s	remaining: 10s
811:	learn: 0.4128299	total: 43.1s	remaining: 9.98s
812:	learn: 0.4128024	total: 43.1s	remaining: 9.92s
813:	learn: 0.4127827	total: 43.2s	remaining: 9.87s
814:	learn: 0.4127705	total: 43.3s	remaining: 9.82s
815:	learn: 0.4127499	total: 43.3s	remaining: 9.77s
816:	learn: 0.4127265	total: 43.4s	remaining: 9.72s
817:	learn: 0.4127069	total: 43.4s	remaining: 9.66s
818:	learn: 0.4126859	total: 43.5s	remaining: 9.61s
819:	learn: 0.4126746	total: 43.5s	remaining: 9.56s
820:	learn: 0.4126533	total: 43.6s	remaining: 9.5s
821:	learn: 0.4126415	total: 43.7s	remaining: 9.45s
822:	learn: 0.4126275	total: 43.7s	remaining: 9.4s
823:	learn: 0.4126161	total: 43.8s	remaining: 9.35s
824:	learn: 0.4126010	total: 43.8s	remaining: 9.29s
825:	learn: 0.4125848	total: 43.9s	remaining: 9.24s
826:	learn: 0.4125623	total: 43.9s	remaining: 9.18s
827:	learn: 0.412548

969:	learn: 0.4100298	total: 51.4s	remaining: 1.59s
970:	learn: 0.4100167	total: 51.5s	remaining: 1.54s
971:	learn: 0.4099934	total: 51.5s	remaining: 1.48s
972:	learn: 0.4099772	total: 51.6s	remaining: 1.43s
973:	learn: 0.4099604	total: 51.6s	remaining: 1.38s
974:	learn: 0.4099433	total: 51.7s	remaining: 1.32s
975:	learn: 0.4099350	total: 51.7s	remaining: 1.27s
976:	learn: 0.4099196	total: 51.8s	remaining: 1.22s
977:	learn: 0.4099055	total: 51.8s	remaining: 1.17s
978:	learn: 0.4098904	total: 51.9s	remaining: 1.11s
979:	learn: 0.4098714	total: 51.9s	remaining: 1.06s
980:	learn: 0.4098525	total: 52s	remaining: 1.01s
981:	learn: 0.4098393	total: 52s	remaining: 954ms
982:	learn: 0.4098168	total: 52.1s	remaining: 901ms
983:	learn: 0.4097987	total: 52.1s	remaining: 848ms
984:	learn: 0.4097731	total: 52.2s	remaining: 795ms
985:	learn: 0.4097551	total: 52.3s	remaining: 742ms
986:	learn: 0.4097425	total: 52.3s	remaining: 689ms
987:	learn: 0.4097242	total: 52.4s	remaining: 636ms
988:	learn: 0.40

StackingCVClassifier(classifiers=[LogisticRegression(C=1.0, class_weight=None,
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None,
                                                     max_iter=100,
                                                     multi_class='warn',
                                                     n_jobs=None, penalty='l2',
                                                     random_state=None,
                                                     solver='warn', tol=0.0001,
                                                     verbose=0,
                                                     warm_start=False),
                                  AdaBoostClassifier(algorithm='SAMME.R',
                                                     base_e

In [76]:
pr = stack.predict_proba(x_test)

print(roc_auc_score(y_test, pr[:,1]))

In [78]:
stack.fit(x_dum.drop(original, axis=1), y)
pred = stack.predict_proba(X_dum.drop(original, axis=1))


[23:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:35:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:36:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:36:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

39:	learn: 0.4336933	total: 4.21s	remaining: 1m 41s
40:	learn: 0.4333840	total: 4.31s	remaining: 1m 40s
41:	learn: 0.4332877	total: 4.43s	remaining: 1m 41s
42:	learn: 0.4331570	total: 4.55s	remaining: 1m 41s
43:	learn: 0.4330629	total: 4.66s	remaining: 1m 41s
44:	learn: 0.4329621	total: 4.79s	remaining: 1m 41s
45:	learn: 0.4328853	total: 4.92s	remaining: 1m 41s
46:	learn: 0.4328197	total: 5.02s	remaining: 1m 41s
47:	learn: 0.4327642	total: 5.14s	remaining: 1m 41s
48:	learn: 0.4326942	total: 5.25s	remaining: 1m 41s
49:	learn: 0.4326064	total: 5.38s	remaining: 1m 42s
50:	learn: 0.4325491	total: 5.49s	remaining: 1m 42s
51:	learn: 0.4324991	total: 5.59s	remaining: 1m 41s
52:	learn: 0.4324122	total: 5.72s	remaining: 1m 42s
53:	learn: 0.4323648	total: 5.82s	remaining: 1m 41s
54:	learn: 0.4323006	total: 5.93s	remaining: 1m 41s
55:	learn: 0.4322005	total: 6.07s	remaining: 1m 42s
56:	learn: 0.4321472	total: 6.17s	remaining: 1m 42s
57:	learn: 0.4320197	total: 6.27s	remaining: 1m 41s
58:	learn: 0

196:	learn: 0.4262795	total: 19.6s	remaining: 1m 19s
197:	learn: 0.4262368	total: 19.7s	remaining: 1m 19s
198:	learn: 0.4262163	total: 19.8s	remaining: 1m 19s
199:	learn: 0.4261902	total: 19.8s	remaining: 1m 19s
200:	learn: 0.4261671	total: 19.9s	remaining: 1m 19s
201:	learn: 0.4261486	total: 20s	remaining: 1m 18s
202:	learn: 0.4261189	total: 20.1s	remaining: 1m 18s
203:	learn: 0.4260876	total: 20.1s	remaining: 1m 18s
204:	learn: 0.4260626	total: 20.2s	remaining: 1m 18s
205:	learn: 0.4260366	total: 20.3s	remaining: 1m 18s
206:	learn: 0.4260127	total: 20.3s	remaining: 1m 17s
207:	learn: 0.4259869	total: 20.4s	remaining: 1m 17s
208:	learn: 0.4259569	total: 20.5s	remaining: 1m 17s
209:	learn: 0.4259321	total: 20.5s	remaining: 1m 17s
210:	learn: 0.4258945	total: 20.6s	remaining: 1m 17s
211:	learn: 0.4258481	total: 20.7s	remaining: 1m 16s
212:	learn: 0.4258188	total: 20.8s	remaining: 1m 16s
213:	learn: 0.4257798	total: 20.8s	remaining: 1m 16s
214:	learn: 0.4257421	total: 20.9s	remaining: 1m

354:	learn: 0.4220324	total: 30.7s	remaining: 55.8s
355:	learn: 0.4220100	total: 30.8s	remaining: 55.7s
356:	learn: 0.4219895	total: 30.9s	remaining: 55.6s
357:	learn: 0.4219705	total: 30.9s	remaining: 55.5s
358:	learn: 0.4219408	total: 31s	remaining: 55.4s
359:	learn: 0.4219203	total: 31.1s	remaining: 55.2s
360:	learn: 0.4218978	total: 31.1s	remaining: 55.1s
361:	learn: 0.4218818	total: 31.2s	remaining: 55s
362:	learn: 0.4218583	total: 31.3s	remaining: 54.9s
363:	learn: 0.4218332	total: 31.3s	remaining: 54.8s
364:	learn: 0.4218108	total: 31.4s	remaining: 54.7s
365:	learn: 0.4217860	total: 31.5s	remaining: 54.5s
366:	learn: 0.4217682	total: 31.6s	remaining: 54.4s
367:	learn: 0.4217513	total: 31.6s	remaining: 54.3s
368:	learn: 0.4217350	total: 31.7s	remaining: 54.2s
369:	learn: 0.4217105	total: 31.8s	remaining: 54.1s
370:	learn: 0.4216816	total: 31.8s	remaining: 54s
371:	learn: 0.4216419	total: 31.9s	remaining: 53.9s
372:	learn: 0.4216200	total: 32s	remaining: 53.8s
373:	learn: 0.421596

513:	learn: 0.4185001	total: 41.9s	remaining: 39.6s
514:	learn: 0.4184783	total: 42s	remaining: 39.5s
515:	learn: 0.4184513	total: 42s	remaining: 39.4s
516:	learn: 0.4184321	total: 42.1s	remaining: 39.3s
517:	learn: 0.4184193	total: 42.2s	remaining: 39.2s
518:	learn: 0.4183953	total: 42.2s	remaining: 39.1s
519:	learn: 0.4183783	total: 42.3s	remaining: 39s
520:	learn: 0.4183575	total: 42.4s	remaining: 39s
521:	learn: 0.4183411	total: 42.4s	remaining: 38.9s
522:	learn: 0.4183293	total: 42.5s	remaining: 38.8s
523:	learn: 0.4183075	total: 42.6s	remaining: 38.7s
524:	learn: 0.4182913	total: 42.6s	remaining: 38.6s
525:	learn: 0.4182698	total: 42.7s	remaining: 38.5s
526:	learn: 0.4182519	total: 42.8s	remaining: 38.4s
527:	learn: 0.4182342	total: 42.9s	remaining: 38.3s
528:	learn: 0.4182166	total: 42.9s	remaining: 38.2s
529:	learn: 0.4181992	total: 43s	remaining: 38.1s
530:	learn: 0.4181817	total: 43.1s	remaining: 38s
531:	learn: 0.4181595	total: 43.2s	remaining: 38s
532:	learn: 0.4181345	tota

...

285:	learn: 0.4237809	total: 17.2s	remaining: 42.9s
286:	learn: 0.4237464	total: 17.3s	remaining: 42.9s
287:	learn: 0.4237219	total: 17.3s	remaining: 42.8s
288:	learn: 0.4236968	total: 17.4s	remaining: 42.8s
289:	learn: 0.4236743	total: 17.4s	remaining: 42.7s
290:	learn: 0.4236521	total: 17.5s	remaining: 42.5s
291:	learn: 0.4236291	total: 17.5s	remaining: 42.4s
292:	learn: 0.4236128	total: 17.6s	remaining: 42.4s
293:	learn: 0.4235937	total: 17.6s	remaining: 42.3s
294:	learn: 0.4235622	total: 17.7s	remaining: 42.3s
295:	learn: 0.4235151	total: 17.8s	remaining: 42.2s
296:	learn: 0.4234963	total: 17.8s	remaining: 42.1s
297:	learn: 0.4234739	total: 17.8s	remaining: 42s
298:	learn: 0.4234436	total: 17.9s	remaining: 41.9s
299:	learn: 0.4234183	total: 17.9s	remaining: 41.9s
300:	learn: 0.4234022	total: 18s	remaining: 41.8s
301:	learn: 0.4233816	total: 18s	remaining: 41.7s
302:	learn: 0.4233548	total: 18.1s	remaining: 41.7s
303:	learn: 0.4233331	total: 18.2s	remaining: 41.6s
304:	learn: 0.4233

444:	learn: 0.4201699	total: 27.2s	remaining: 34s
445:	learn: 0.4201570	total: 27.3s	remaining: 33.9s
446:	learn: 0.4201376	total: 27.3s	remaining: 33.8s
447:	learn: 0.4201188	total: 27.4s	remaining: 33.7s
448:	learn: 0.4200987	total: 27.5s	remaining: 33.7s
449:	learn: 0.4200840	total: 27.5s	remaining: 33.6s
450:	learn: 0.4200560	total: 27.5s	remaining: 33.5s
451:	learn: 0.4200334	total: 27.6s	remaining: 33.5s
452:	learn: 0.4200161	total: 27.7s	remaining: 33.4s
453:	learn: 0.4199999	total: 27.7s	remaining: 33.3s
454:	learn: 0.4199732	total: 27.8s	remaining: 33.3s
455:	learn: 0.4199524	total: 27.8s	remaining: 33.2s
456:	learn: 0.4199307	total: 27.9s	remaining: 33.1s
457:	learn: 0.4199061	total: 27.9s	remaining: 33.1s
458:	learn: 0.4198892	total: 28s	remaining: 33s
459:	learn: 0.4198750	total: 28s	remaining: 32.9s
460:	learn: 0.4198529	total: 28.1s	remaining: 32.8s
461:	learn: 0.4198275	total: 28.1s	remaining: 32.8s
462:	learn: 0.4198061	total: 28.2s	remaining: 32.7s
463:	learn: 0.419789

603:	learn: 0.4171402	total: 36s	remaining: 23.6s
604:	learn: 0.4171217	total: 36.1s	remaining: 23.6s
605:	learn: 0.4171040	total: 36.1s	remaining: 23.5s
606:	learn: 0.4170837	total: 36.2s	remaining: 23.4s
607:	learn: 0.4170631	total: 36.3s	remaining: 23.4s
608:	learn: 0.4170391	total: 36.3s	remaining: 23.3s
609:	learn: 0.4170248	total: 36.4s	remaining: 23.3s
610:	learn: 0.4170108	total: 36.5s	remaining: 23.2s
611:	learn: 0.4169986	total: 36.5s	remaining: 23.2s
612:	learn: 0.4169844	total: 36.6s	remaining: 23.1s
613:	learn: 0.4169663	total: 36.7s	remaining: 23s
614:	learn: 0.4169530	total: 36.7s	remaining: 23s
615:	learn: 0.4169388	total: 36.8s	remaining: 22.9s
616:	learn: 0.4169198	total: 36.8s	remaining: 22.9s
617:	learn: 0.4168980	total: 36.9s	remaining: 22.8s
618:	learn: 0.4168855	total: 37s	remaining: 22.8s
619:	learn: 0.4168700	total: 37s	remaining: 22.7s
620:	learn: 0.4168519	total: 37.1s	remaining: 22.6s
621:	learn: 0.4168293	total: 37.1s	remaining: 22.6s
622:	learn: 0.4168105	

764:	learn: 0.4143722	total: 45.1s	remaining: 13.8s
765:	learn: 0.4143507	total: 45.1s	remaining: 13.8s
766:	learn: 0.4143392	total: 45.2s	remaining: 13.7s
767:	learn: 0.4143277	total: 45.2s	remaining: 13.7s
768:	learn: 0.4143117	total: 45.3s	remaining: 13.6s
769:	learn: 0.4142946	total: 45.3s	remaining: 13.5s
770:	learn: 0.4142786	total: 45.4s	remaining: 13.5s
771:	learn: 0.4142522	total: 45.4s	remaining: 13.4s
772:	learn: 0.4142283	total: 45.5s	remaining: 13.4s
773:	learn: 0.4142159	total: 45.6s	remaining: 13.3s
774:	learn: 0.4141969	total: 45.6s	remaining: 13.2s
775:	learn: 0.4141784	total: 45.7s	remaining: 13.2s
776:	learn: 0.4141634	total: 45.7s	remaining: 13.1s
777:	learn: 0.4141528	total: 45.8s	remaining: 13.1s
778:	learn: 0.4141386	total: 45.8s	remaining: 13s
779:	learn: 0.4141239	total: 45.9s	remaining: 12.9s
780:	learn: 0.4141081	total: 45.9s	remaining: 12.9s
781:	learn: 0.4140910	total: 46s	remaining: 12.8s
782:	learn: 0.4140741	total: 46.1s	remaining: 12.8s
783:	learn: 0.41

925:	learn: 0.4117207	total: 54.6s	remaining: 4.37s
926:	learn: 0.4117003	total: 54.7s	remaining: 4.31s
927:	learn: 0.4116893	total: 54.7s	remaining: 4.25s
928:	learn: 0.4116764	total: 54.8s	remaining: 4.19s
929:	learn: 0.4116645	total: 54.9s	remaining: 4.13s
930:	learn: 0.4116556	total: 54.9s	remaining: 4.07s
931:	learn: 0.4116435	total: 55s	remaining: 4.01s
932:	learn: 0.4116282	total: 55s	remaining: 3.95s
933:	learn: 0.4116133	total: 55.1s	remaining: 3.89s
934:	learn: 0.4116008	total: 55.1s	remaining: 3.83s
935:	learn: 0.4115897	total: 55.2s	remaining: 3.77s
936:	learn: 0.4115771	total: 55.2s	remaining: 3.71s
937:	learn: 0.4115652	total: 55.3s	remaining: 3.65s
938:	learn: 0.4115512	total: 55.4s	remaining: 3.6s
939:	learn: 0.4115332	total: 55.5s	remaining: 3.54s
940:	learn: 0.4115159	total: 55.5s	remaining: 3.48s
941:	learn: 0.4115057	total: 55.6s	remaining: 3.42s
942:	learn: 0.4114908	total: 55.6s	remaining: 3.36s
943:	learn: 0.4114758	total: 55.7s	remaining: 3.31s
944:	learn: 0.411

In [79]:
sub = pd.read_csv('sample_submission_eyYijxG.csv')
sub['Is_Lead'] = pred

In [80]:
sub

,ID,Is_Lead
0,VBENBARO,0.960889
1,CCMEWNKY,0.750949
2,VK3KGA9M,0.925694
3,TT8RPZVC,0.959105
4,SHQZEYTZ,0.971480
...,...,...
105307,DBENJOYI,0.009456
105308,CWQ72DWS,0.315413
105309,HDESC8GU,0.911601
105310,2PW4SFCA,0.590959


In [81]:
sub.to_csv('Sub_5.csv', index=False)

In [58]:
for item in pr[:,1]:
    print(item, ':', abs(1-np.exp(item))%1)

0.13744504693661166 : 0.1473386544658446
0.2737109272111689 : 0.31483466437433294
0.19818677051560551 : 0.2191900813184342
0.032471831210807885 : 0.03300479423985059
0.029464972679585883 : 0.029903360085093977
0.11776753847977275 : 0.12498256581140366
0.32085165217053385 : 0.3783010977506116
0.3040110125102689 : 0.3552839810862496
0.07343343599062847 : 0.07619689830722542
0.558169080858485 : 0.7474700930737359
0.28379779530387067 : 0.3281643425315186
0.44517976559383604 : 0.5607707434950489
0.5816207984019677 : 0.7889355862027174
0.6826300841746665 : 0.9790760298002668
0.41259894401861663 : 0.5107390135019871
0.4319983876773108 : 0.5403326316458741
0.23777674440038216 : 0.26842597800914914
0.13618168585845838 : 0.1458900667037022
0.11510090342798289 : 0.12198664415892257
0.5543290410447692 : 0.7407726058818305
0.29797155134079273 : 0.3471234634780951
0.06204309878306714 : 0.0640082011984926
0.579775054868684 : 0.7856367152843959
0.347071957516474 : 0.4149185357312186
0.0773690307140703

0.06122788746485266 : 0.06314116312784401
0.3692784260318149 : 0.44669034385716766
0.5197982624944527 : 0.6816883558618714
0.18069212989989852 : 0.19804627988145085
0.06495969133960401 : 0.06711600950043217
0.03086026648046869 : 0.03134138085636273
0.1563362031616557 : 0.16921923221989
0.1036221796595564 : 0.10918130448071595
0.732758794225573 : 0.08081323200172807
0.08906145768042377 : 0.09314783659233461
0.07101873110728578 : 0.07360133542181568
0.1502715894614363 : 0.16214982751735563
0.059638604815900026 : 0.06145287326415061
0.2810370668355909 : 0.32450269814912014
0.6575731744155612 : 0.9301026237209247
0.08921990377179581 : 0.09332105531688706
0.02510790609932655 : 0.02542576424912424
0.09231349945228369 : 0.09670858573408547
0.5705775865243802 : 0.7692886736633635
0.32187094853619463 : 0.37970671129702604
0.14989916145580737 : 0.16171709096145936
0.4631054569459101 : 0.5890009047630829
0.06306459461435136 : 0.06509563645120053
0.18431817376041165 : 0.20239833384002415
0.3939334

0.09488412229343304 : 0.099531436562593
0.07021882279010316 : 0.07274289616639651
0.025272920948712564 : 0.025594988689056963
0.23367205634777422 : 0.2632301559610726
0.5291665969356332 : 0.6975170030683053
0.7062496375957128 : 0.026377340699764495
0.05407781352489239 : 0.055566736327976596
0.07359896366189689 : 0.07637505341802187
0.21119845380188768 : 0.23515745243270247
0.12854690293182394 : 0.1371747568705015
0.1553267111904472 : 0.16803951035244347
0.4478461000066704 : 0.5649378331975417
0.18616811163613775 : 0.20462475479477948
0.2302463699867305 : 0.2589101293977192
0.32587792684778916 : 0.385246257173794
0.19427583706269094 : 0.21443122190091146
0.2732974675582519 : 0.3142911456596298
0.0672885540684078 : 0.06960407225605514
0.07989047682102618 : 0.08316842912851952
0.26327900126750153 : 0.30118970188910676
0.28241634748978195 : 0.32633081955352394
0.32912785163414177 : 0.38975552674474656
0.05739846465194666 : 0.059077731369235
0.4376666992464004 : 0.5490885089988562
0.2613302

0.08618518828262817 : 0.09000816635590891
0.08931184726879753 : 0.09342158369945319
0.03683064833415754 : 0.03751730067611603
0.3994602245975803 : 0.49101966465281
0.28945333702717796 : 0.33569711225479093
0.053828910150270214 : 0.055304034900157895
0.051319356541780145 : 0.05265901315142152
0.38100010719705607 : 0.46374776231917103
0.23048950334544427 : 0.2592162496583139
0.32971400311161514 : 0.39057037278835094
0.0480666954733523 : 0.049240632591512234
0.42894951819435007 : 0.535643510383351
0.06413252504367384 : 0.06623369206545537
0.22354285110808725 : 0.25049922440581507
0.20933728347011316 : 0.23286075196552214
0.0781267389845637 : 0.08125968774131098
0.07410397180746611 : 0.07691876886649518
0.12786986811299378 : 0.13640511053308857
0.06343639422670364 : 0.0654917122217975
0.6180903467277993 : 0.8553815212185858
0.21594028687645375 : 0.24102827111360958
0.08891919774973768 : 0.09299233651784844
0.9953442390043931 : 0.7056555731678249
0.40839920554573966 : 0.5044076091716858
0.0

0.1935091175440426 : 0.21350045064486545
0.28800626305607824 : 0.3337656575463339
0.06062426582510049 : 0.06249962175927459
0.23529272512535165 : 0.2652790935098166
0.5448446731145173 : 0.724340525130978
0.2837418876400073 : 0.328090090041562
0.32225209428235396 : 0.3802326808701497
0.2104593676902341 : 0.23424490198212622
0.1344499006215101 : 0.14390734851958675
0.10623048206357837 : 0.1120781610403192
0.22912817418770642 : 0.25750320813299843
0.14303176633102138 : 0.15376645201091987
0.32527208908883565 : 0.3844072768543889
0.29986357871249797 : 0.34967467065991076
0.1042581210369756 : 0.10988690310319504
0.3974772760726343 : 0.4880659788793902
0.036235029897483076 : 0.03689952024243648
0.35415750923563705 : 0.42497961625469793
0.0702308595263239 : 0.07275580856738162
0.10655092922764155 : 0.11243458043698373
0.11477958861113587 : 0.12162619113839912
0.11374012301854884 : 0.12046090504728824
0.05974308001681773 : 0.06156377455946638
0.40353577744907065 : 0.4971087939544241
0.07877939

...

0.5932880149162568 : 0.8099297184102643
0.6179077545602528 : 0.8550427740122353
0.3314940847517371 : 0.3930479060297556
0.11245315813261837 : 0.11901983870622468
0.4256026940407174 : 0.5305125725541606
0.4076670625230415 : 0.5033065707453204
0.20506351098040784 : 0.22760302875928096
0.22761901548171334 : 0.25560686751802364
0.9961665531420747 : 0.707881387031017
0.7506205664955505 : 0.11831416360968916
0.16117647421370979 : 0.17489228871286966
0.0625465448997685 : 0.06454400685884698
0.17185812244804954 : 0.1875093403433843
0.05366624515312285 : 0.05513238783319463
0.05792204731338563 : 0.05963239129898601
0.06668657572285533 : 0.0689603875278142
0.0501645969847206 : 0.05144414666999908
0.33500165548425725 : 0.3979426994926829
0.12417396033540766 : 0.13221281398941853
0.3465837947491985 : 0.4142279937455031
0.07422345869532397 : 0.07704745422661641
0.026312751845492924 : 0.02666198869952696
0.7578276946473096 : 0.13363627313337423
0.048858766435513866 : 0.05007203485023459
0.0730821510

0.07702806968451459 : 0.08007239325843751
0.06307900087574479 : 0.06511098060787379
0.04909025405383494 : 0.050315141661670504
0.031635270717483115 : 0.03214098460445891
0.23294840656207735 : 0.26231635040689993
0.11578158961937988 : 0.12275062496062628
0.07647123955487657 : 0.07947114382011766
0.23679826737816345 : 0.26718545934376126
0.4698545542702279 : 0.599761497816983
0.16425116079331306 : 0.17851027349981208
0.10914819318795721 : 0.11532762205100311
0.40554134182415674 : 0.500114354932784
0.13517434593619326 : 0.14473634708446004
0.474194801497331 : 0.6067199479954992
0.5858700897032941 : 0.7965534684655462
0.3680266470023379 : 0.4448805401960383
0.11301741849370553 : 0.11965143542021384
0.5669622379319288 : 0.7629036273862047
0.2560192814194493 : 0.29177763500599707
0.3762953963478775 : 0.4568774264855979
0.1397222247688644 : 0.14995432565980527
0.021422603546427714 : 0.02165371490303003
0.023087071394383367 : 0.02335564067066498
0.07584859358186448 : 0.07879922466502132
0.3650

0.03208245998731397 : 0.03260265019598241
0.4175101831510705 : 0.5181768636441226
0.058677240971094884 : 0.060432921199903955
0.4358954340854791 : 0.5463470910974324
0.2473106152800452 : 0.2805768177353143
0.0737428155663851 : 0.0765299031568476
0.23031857875803102 : 0.25900103703346566
0.18312229125752527 : 0.20096126616439625
0.3268295483791677 : 0.3865651147654947
0.2777393323127409 : 0.3201420339840759
0.05148022180958117 : 0.052828363046368176
0.06699891685335502 : 0.06929431997141777
0.05009718252846427 : 0.05137326652376739
0.13653384564274562 : 0.14629367416539663
0.1291368720941306 : 0.13784585285271045
0.14855899828545488 : 0.16016124327952808
0.3522689631420471 : 0.42229101613869613
0.08533168077813691 : 0.08907823311492002
0.11658683274953881 : 0.12365507629099604
0.01836471312176097 : 0.018534381511418063
0.03502342678928559 : 0.03564397032850497
0.06258732377752091 : 0.06458741865390083
0.04202092278821122 : 0.042916299239340816
0.026327578348201163 : 0.026677210619126424

0.020604505850654854 : 0.020818244147936982
0.02166605124092943 : 0.02190246442202204
0.2579476195280516 : 0.2942710223216738
0.16972084252835773 : 0.18497401079539633
0.026364371668173965 : 0.026714986177185374
0.571016509785951 : 0.7700654260735484
0.1647067917695141 : 0.17904736163389345
0.07211809853147329 : 0.07478226677633293
0.4547913898951419 : 0.5758446116329923
0.37054102647389414 : 0.4485180893381897
0.06109912403555661 : 0.06300427823892707
0.14581338642343847 : 0.1569802596795702
0.10284336696854293 : 0.10831779630336191
0.06720256648787185 : 0.06951210354389059
0.06096729996589503 : 0.06286415792471978
0.04776910832244299 : 0.04892843851581774
0.06957311187174295 : 0.07205043795372212
0.2950145993811974 : 0.34314596764312255
0.07624101727140412 : 0.07922265411347618
0.16341279942205092 : 0.1775226700530088
0.07040965645218172 : 0.07294763115628311
0.0580697375320658 : 0.059788900195709704
0.34715755952676713 : 0.41503966078647836
0.1509247722450195 : 0.16290917174492447
0

0.3725713255603615 : 0.45146200179182516
0.13224326496907496 : 0.14138594471413657
0.13429506631535865 : 0.14373024613009378
0.7749360147515411 : 0.17045324575011023
0.09680216657605764 : 0.10164241037150967
0.999107089639358 : 0.7158557297567949
0.1933022904728266 : 0.21324949185416986
0.14705162315497458 : 0.15841376245950856
0.07211992425650021 : 0.07478422903500714
0.0578386246277508 : 0.059543997606189114
0.28012853971550167 : 0.32329989799823355
0.11409648672752423 : 0.12086026780614345
0.06565206846849685 : 0.06785511205861305
0.1104028186044597 : 0.11672781861078252
0.28703736136212005 : 0.3324739955891711
0.10962337228774054 : 0.11585772836415842
0.29022211173103707 : 0.3367243572160874
0.06841261541823984 : 0.07080704883637567
0.6118788038241239 : 0.843892458639854
0.07911736485061051 : 0.08233134227261218
0.08141724796402727 : 0.08482344252097662
0.059022641622338994 : 0.06079925868444791
0.08126377900938411 : 0.08465696857589844
0.0829307288990182 : 0.08646654520771446
0.13